Groceries Dataset

In [1]:
import pandas as pd
df = pd.read_csv('./data/groceries/Groceries_dataset.csv', header=None, names = ['userId', 'date', 'item'])
df = df.set_index('userId')['item']
df.to_csv('./data/groceries/transactions.csv')
display(df)

userId
1808           tropical fruit
2552               whole milk
2300                pip fruit
1187         other vegetables
3037               whole milk
                ...          
4471            sliced cheese
2022                    candy
1097                 cake bar
1510    fruit/vegetable juice
1521                 cat food
Name: item, Length: 38765, dtype: object

Movielens Dataset

In [2]:
import pandas as pd
df = pd.read_csv('./data/movielens/ratings.csv', header=None, names = ['userId', 'movieId', 'rating', 'timestamp'])
df = df[df['rating'] >= 3.0].set_index('userId')['movieId']
df.to_csv('./data/movielens/transactions.csv')
display(df)

userId
1           1
1           3
1           6
1          47
1          50
        ...  
610    166534
610    168248
610    168250
610    168252
610    170875
Name: movieId, Length: 81763, dtype: int64

Amazon-Reviews

In [2]:
from pyspark import SparkContext
import json
import csv
import sys

business_file_path = "./data/yelp_dataset/business.json"
review_file_path = "./data/yelp_dataset/review.json"
output_file_path = "./data/yelp_dataset/user_business.csv"
state = "NV"

sc = SparkContext.getOrCreate()
business_list = sc.textFile(business_file_path).map(lambda row: json.loads(row))\
    .map(lambda json_data: (json_data["business_id"], json_data["state"]))\
    .filter(lambda kv: kv[1] == state).map(lambda line: line[0]).collect()

user_list = sc.textFile(review_file_path).map(lambda row: json.loads(row))\
    .map(lambda json_data: (json_data["user_id"], json_data["business_id"]))\
    .filter(lambda kv: kv[1] in business_list).collect()

with open(output_file_path, mode='w', newline='') as result_file:
    # write output to csv file
    result_writer = csv.writer(result_file)
    result_writer.writerow(["user_id", "business_id"])
    result_writer.writerows(i for i in user_list)
    result_file.close()